In [1]:
!wget https://dumps.wikimedia.org/zhwiki/20231201/zhwiki-20231201-pages-articles-multistream1.xml-p1p187712.bz2
#使用wget工具下載一個維基百科的數據檔案。

--2023-12-28 06:11:21--  https://dumps.wikimedia.org/zhwiki/20231201/zhwiki-20231201-pages-articles-multistream1.xml-p1p187712.bz2
Resolving dumps.wikimedia.org (dumps.wikimedia.org)... 208.80.154.142, 2620:0:861:2:208:80:154:142
Connecting to dumps.wikimedia.org (dumps.wikimedia.org)|208.80.154.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 230609687 (220M) [application/octet-stream]
Saving to: ‘zhwiki-20231201-pages-articles-multistream1.xml-p1p187712.bz2’

zhwiki-20231201-pag 100%[===================>] 219.93M  4.23MB/s    in 54s     

2023-12-28 06:12:15 (4.10 MB/s) - ‘zhwiki-20231201-pages-articles-multistream1.xml-p1p187712.bz2’ saved [230609687/230609687]



In [2]:
!pip install opencc-python-reimplemented
#使用pip安裝一個名為opencc-python-reimplemented的Python套件。

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 481.8/481.8 kB 4.8 MB/s eta 0:00:00


In [5]:
from gensim.corpora import WikiCorpus
#從Gensim中的corpora模組中導入WikiCorpus類別。
wiki_corpus = WikiCorpus('zhwiki-20231201-pages-articles-multistream1.xml-p1p187712.bz2', dictionary={})


In [6]:
wiki_corpus
#創建了一個名為wiki_corpus的WikiCorpus實例。

In [7]:
next(iter(wiki_corpus.get_texts()))[:10]
#使用wiki_corpus.get_texts()取得維基百科語料庫的文本，然後使用iter()將其轉換為可迭代對象，最後使用next(...)取得迭代器的下一個元素。

['歐幾里得',
 '西元前三世紀的古希臘數學家',
 '現在被認為是幾何之父',
 '此畫為拉斐爾的作品',
 '雅典學院',
 '数学',
 '是研究數量',
 '屬於形式科學的一種',
 '數學利用抽象化和邏輯推理',
 '從計數']

In [9]:
text_num = 0
#初始化一個變數text_num，用於計算處理的文本數量。
with open('wiki_text.txt', 'w', encoding='utf-8') as f:
#打開一個檔案wiki_text.txt以進行寫入，使用 UTF-8 編碼。
    for text in wiki_corpus.get_texts():
    #遍歷 wiki_corpus 中的文本。
        f.write(' '.join(text)+'\n')
        #將處理過的文本以空格分隔的詞彙序列的形式寫入文件。
        text_num += 1
        #計算處理的文本數量。
        if text_num % 10000 == 0:
        #當處理的文本數量達到10000篇時，執行下面的代碼塊。
            print('{} articles processed.'.format(text_num))
            #列印目前處理的文本數量。

    print('{} articles processed.'.format(text_num))
    #在所有文本處理完成後，再次列印總處理的文本數量。

10000 articles processed.
20000 articles processed.
30000 articles processed.
32705 articles processed.


In [10]:
import jieba
from opencc import OpenCC


# Initial
cc = OpenCC('s2t')
#創建一個OpenCC的實例，並指定轉換方式為從簡體中文(s)到繁體中文(t)。
train_data = open('wiki_text.txt', 'r', encoding='utf-8').read()
#讀取'wiki_text.txt'檔案中的內容
train_data = cc.convert(train_data)
#使用OpenCC將簡體中文轉換為繁體中文。
train_data = jieba.lcut(train_data)
#使用jieba分詞，將文本拆分為一個個詞語。
train_data = [word for word in train_data if word != '']
#去除空白詞語，保留非空的詞語。
train_data = ' '.join(train_data)
#將分詞後的詞語列表組合成一個以空格分隔的字串。
open('seg.txt', 'w', encoding='utf-8').write(train_data)
#將處理完的文本寫入'seg.txt'檔案中，用作後續的分析和處理。

Building prefix dict from the default dictionary ...
DEBUG:jieba:Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
DEBUG:jieba:Dumping model to file cache /tmp/jieba.cache
Loading model cost 1.577 seconds.
DEBUG:jieba:Loading model cost 1.577 seconds.
Prefix dict has been built successfully.
DEBUG:jieba:Prefix dict has been built successfully.


135539442

In [11]:
from gensim.models import word2vec
#導入Gensim中的Word2Vec模型。

# Settings
seed = 666
#設定亂數種子，確保結果的可重複性。
sg = 0
#設定Skip-gram模型。
window_size = 10
#設定窗口大小。
#vector_size = 100
min_count = 1
#詞頻閾值，詞頻低於此值的詞將被忽略。
workers = 8
#設定模型訓練的並行工作數。
#epochs = 5
batch_words = 10000
#每一個訓練批次的單詞數。
train_data = word2vec.LineSentence('seg.txt')
#從'seg.txt'中讀取分詞後的文本數據，使用LineSentence來處理每一行的分詞。
model = word2vec.Word2Vec(
    train_data,
    min_count=min_count,
    #size=vector_size,
    workers=workers,
    #iter=epochs,
    window=window_size,
    sg=sg,
    seed=seed,
    batch_words=batch_words
)
#設定參數建立Word2Vec模型。
model.save('word2vec.model')
#將訓練好的Word2Vec模型儲存到'word2vec.model'檔案中。


In [12]:
from gensim.models import word2vec
#導入Gensim中的Word2Vec模型。
string = '電腦'
#設定一個詞語'電腦'。
model = word2vec.Word2Vec.load('word2vec.model')
#從'word2vec.model'中載入訓練好的Word2Vec模型。
print(string)
#列印出設定的詞語。
for item in model.wv.most_similar(string):
#對於模型中與輸入詞語最相似的詞，使用most_similar方法迭代取出每個相似詞。
    print(item)
    #列印出相似詞及其相似度。

電腦
('計算機', 0.7747930288314819)
('軟體', 0.7732110619544983)
('程式', 0.7476573586463928)
('pda', 0.7414882779121399)
('遊戲機', 0.7409443855285645)
('硬體', 0.7400156259536743)
('pc', 0.7379648089408875)
('ibm', 0.728009045124054)
('家用', 0.7150958776473999)
('手機', 0.7146145701408386)
